In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.graph_objects as go

from datetime import datetime

### 1 - Import test data

In [ ]:
dataF.columns = dataF.columns.droplevel(1)
dataF.reset_index(inplace=True)

In [ ]:
display(dataF)

In [ ]:
def calculate_atr(df, n):
    # Calculate the True Range (TR)
    df['TR'] = df[['High', 'Low', 'Close']].max(axis=1) - df[['High', 'Low', 'Close']].min(axis=1)
    df['TR'] = df['TR'].combine(df['High'] - df['Close'].shift(), max)
    df['TR'] = df['TR'].combine(df['Low'] - df['Close'].shift(), max)
    
    # Calculate the ATR using a rolling window
    df['ATR'] = df['TR'].rolling(window=n).mean()
    return df

def calculate_chandelier_exit(df, n=22, multiplier=3):
    # Calculate the ATR
    df = calculate_atr(df, n)
    
    # Calculate the highest high and lowest low over the lookback period
    df['Highest_High'] = df['High'].rolling(window=n).max()
    df['Lowest_Low'] = df['Low'].rolling(window=n).min()
    
    # Calculate the Chandelier Exit
    df['Chandelier_Exit_Long'] = df['Highest_High'] - df['ATR'] * multiplier
    df['Chandelier_Exit_Short'] = df['Lowest_Low'] + df['ATR'] * multiplier

    df = df.drop(columns=["TR", "Highest_High", 'Lowest_Low'])
    
    return df

In [ ]:
dataF = calculate_chandelier_exit(dataF)

In [ ]:
display(dataF)

In [ ]:
# Calculate the 50-day EMA
dataF['EMA_50'] = dataF['Adj Close'].ewm(span=50, adjust=False).mean()

In [ ]:
fig = go.Figure(data=[go.Candlestick(x= dataF.index, # dataF["Datetime"]
                open=dataF['Open'],
                high=dataF['High'],
                low=dataF['Low'],
                close=dataF['Close'])])

fig.add_trace(go.Scatter(
    x= dataF.index, # dataF["Datetime"]
    y=dataF['EMA_50'],
    mode='lines',
    name='50-day EMA',
    line=dict(color='blue', width=2)
))

fig.add_trace(go.Scatter(
    x= dataF.index, # dataF["Datetime"]
    y=dataF['Chandelier_Exit_Long'],
    mode='lines',
    name='chandelier stop',
    line=dict(color='black', width=2)
))

fig.update_layout(height=800)

fig.show()

In [ ]:
print(dataF["EMA_50"].iloc[-1])

### 2 - Define your signal function

#### We define the buying strategy

In [ ]:
# Note that in between all these functions time can flow (for example between buy and get_swing_high_point_before_pullback) or no time can flow,
# i.e. the functions are executed in the same time frame (for example between pullback and get_swing_high_point_before_pullback there is no time 
# in between, the functions are executed one after the other).

In [ ]:
def price_below_EMA(df):
    # Is True if price is below 50 day EMA 
    open = df.Open.iloc[-1]
    close = df.Close.iloc[-1]
    EMA_50 = df["EMA_50"].iloc[-1]

    if open >= EMA_50 and close <= EMA_50:
        return True
    
    return False

def strategy_start(df):
    # Strategy starts once the price is above EMA and a candle closes above it
    open = df.Open.iloc[-1]
    close = df.Close.iloc[-1]
    EMA_50 = df["EMA_50"].iloc[-1]

    # This needs to be played with
    if close >= EMA_50 and open >= EMA_50: #close - EMA_50 > EMA_50 - open: # close >= EMA_50
        return True
    
    return False

def pullback(df):
    # We consider a Pullback when we have at least 2 opposite candles coming down
    bool1 = (df.Open.iloc[-1] - df.Close.iloc[-1] > 0)
    bool2 = (df.Open.iloc[-2] - df.Close.iloc[-2] > 0)

    # We add this condition which says that the candles must have a certain length
    bool3 = (df.Open.iloc[-2] - df.Close.iloc[-1] >= 0.8*np.abs(df.Close.iloc[-3] - df.Open.iloc[-3]))

    if bool1 and bool2 and bool3:
        return True
    
    return False

def get_swing_high_point_before_pullback(df):
    # We apply this function only if the other one is True

    # Gets high point before pullback
    high_point = df.High.iloc[-3]

    len_high_point_candle = df.Close.iloc[-3] - df.Open.iloc[-3]

    return high_point, len_high_point_candle

def invalid_trade_1(df):
    # Check this condition before buying

    # If the prices closes below the EMA after the pullback then the trade is invalid
    if df.Close.iloc[-1] < df["EMA_50"].iloc[-1]:
        # Trade is invalid
        return True
    # trade is valid
    return False

def invalid_trade_2(df, len_high_point_candle):
    # This functions needs to be called right after get_swing_high_point_before_pullback
    # Calculate tthe mean height of the last 40 candles

    # If the breakcoutcandle is 3 or 4 times bigger than the candles before, then the trade is invalid
    df['AbsDiff'] = (df.Close - df.Open).abs()
    mean_abs_diff = df['AbsDiff'].iloc[:50].mean()

    if len_high_point_candle >= 3*mean_abs_diff:
        return True
    return False


def buy(df, high_point):
    # The body of the candle needs to close above the swing high point
    close_buy = df.Close.iloc[-1]
    stop_loss = df['Chandelier_Exit_Short'].iloc[-1]

    """
    Stop loss signal needs to be below the close_buy otherwise the rest does not make much sense?
    """

    if close_buy >= high_point:
        return True, close_buy, stop_loss
    
    return False, 0, 0

def stop_loss_f(df, stop_loss):
    # Apply this function after the buy signal
    if df.Close.iloc[-1] < stop_loss:
        # We sell the stock again to limit loss
        return True
    return False

def take_profit_target(df, close_buy, stop_loss):
    # Apply this function after the buy signal
    range = np.abs(close_buy - stop_loss)
    if df.High.iloc[-1] >= close_buy + 2*range:
        # We sell the stock to get the profit
        return True
    return False

In [ ]:
# The bots need to be able to do 2 things, see future trading strateegies, and if it is started in one it should be able to work from there
# For this second option I need to go row by row and append to some available data and then run the function below on these partial dataframes
# I will also do this for the testing now but it has to be done once I am connected to the broker also

# I need to implement take proft target

In [ ]:
# Do on running data
def steps(df):
    # Inititalize to True
    global step_1
    global step_2
    global step_3

    # Initialize to None
    global high_point
    global len_high_point_candle
    global close_buy
    global stop_loss

    # Initialize to False
    global is_buy

    if step_1 and price_below_EMA(df):
        print("price below EMA")
        step_1 = False

    if step_2 and not step_1 and strategy_start(df):
        print("strategy start")
        step_2 = False
        
    if step_3 and not step_2 and pullback(df):
        print("pullback")
        step_3 = False
        high_point, len_high_point_candle = get_swing_high_point_before_pullback(df)

    if not is_buy and not step_3 and invalid_trade_1(df): 
        print("Invalid trade1")
        # Start over
        return "invalid1"
        
    if not is_buy and not step_3 and invalid_trade_2(df, len_high_point_candle):
        print("Invalid trade2")
        # Start over
        return "invalid2"

    if not is_buy and not step_3:
        is_buy, close_buy, stop_loss = buy(df, high_point)
        if is_buy:
            return "buy"
        
    return "hold"


In [ ]:
def trading_strategy(df):

    # Initialize to None
    global high_point
    global len_high_point_candle
    global close_buy
    global stop_loss

    global buy_bool

    # Initialize to 1000
    global bank_account

    if buy_bool:
        print("BUY")
        bank_account -= 100*df.Open.iloc[-1]
        buy_bool = False
        return 0
    
    # Implement the stop loss functions here
    if stop_loss_f(df, stop_loss):
        print("sell and loose small amount")
        bank_account += 100*df.Open.iloc[-1]
        return "sell1"

    if take_profit_target(df, close_buy, stop_loss):
        print("sell and make profit")
        bank_account += 100*df.Open.iloc[-1]
        return "sell2"
    
    else:
        return 0
    

In [ ]:
test = dataF[dataF["Datetime"] >= "2024-11-19"]
available_df = pd.DataFrame(columns=test.columns)

In [ ]:
step_1 = True
step_2 = True
step_3 = True

strat = False
buy_bool = True

high_point = None
len_high_point_candle = None
close_buy = None
stop_loss = None 

is_buy = False

In [ ]:
def reinitialize_variables():
    global step_1
    global step_2
    global step_3

    global strat 
    global buy_bool

    global high_point
    global len_high_point_candle
    global close_buy
    global stop_loss

    global is_buy

    step_1 = True
    step_2 = True
    step_3 = True

    strat = False
    buy_bool = True

    high_point = None
    len_high_point_candle = None
    close_buy = None
    stop_loss = None 

    is_buy = False

In [ ]:
step_1 = True
step_2 = True
step_3 = True

strat = False
buy_bool = True

high_point = None
len_high_point_candle = None
close_buy = None
stop_loss = None 

is_buy = False

bank_account = 1000

for index, row in test.iterrows():
    row_df = pd.DataFrame([row])
    available_df = pd.concat([available_df, row_df])
    print(len(available_df))
    print(bank_account)

    if len(available_df) >= 3:
        res = steps(available_df)
        if not strat and (res == "invalid1" or res ==  "invalid2"):
            available_df = pd.DataFrame(columns=test.columns)
            reinitialize_variables()
        elif not strat and res == "buy":
            strat = True
            continue # Go directly to next candle

        if strat:         
            res = trading_strategy(available_df)
            if res == "sell1" or res == "sell2":
                available_df = pd.DataFrame(columns=test.columns)
                reinitialize_variables()
                strat = False
            else:
                continue




In [ ]:
fig2 = go.Figure(data=[go.Candlestick(x= test.index, # dataF["Datetime"]
                open=test['Open'],
                high=test['High'],
                low=test['Low'],
                close=test['Close'])])

fig2.add_trace(go.Scatter(
    x= test.index, # dataF["Datetime"]
    y=test['EMA_50'],
    mode='lines',
    name='50-day EMA',
    line=dict(color='blue', width=2)
))

fig2.update_layout(height=800)

fig2.show()

### Connect to the market

In [ ]:
def signal_generator(df):
    open = df.Open.iloc[-1]
    close = df.Close.iloc[-1]
    previous_open = df.Open.iloc[-2]
    previous_close = df.Close.iloc[-2]
    
    # Bearish Pattern
    if (open>close and 
    previous_open<previous_close and 
    close<previous_open and
    open>=previous_close):
        return 1

    # Bullish Pattern
    elif (open<close and 
        previous_open>previous_close and 
        close>previous_open and
        open<=previous_close):
        return 2
    
    # No clear pattern
    else:
        return 0

signal = []
signal.append(0)
for i in range(1,len(dataF)):
    df = dataF[i-1:i+1]
    signal.append(signal_generator(df))
#signal_generator(data)
dataF["signal"] = signal

In [ ]:
dataF.signal.value_counts()
#dataF.iloc[:, :]

### 3 - Connect to the market and execute trades

In [ ]:
from apscheduler.schedulers.blocking import BlockingScheduler
from oandapyV20 import API
import oandapyV20.endpoints.orders as orders
from oandapyV20.contrib.requests import MarketOrderRequest
from oanda_candles import Pair, Gran, CandleClient
from oandapyV20.contrib.requests import TakeProfitDetails, StopLossDetails

In [ ]:
from config import access_token, accountID
def get_candles(n):
    #access_token='XXXXXXX'#you need token here generated from OANDA account
    client = CandleClient(access_token, real=False)
    collector = client.get_collector(Pair.EUR_USD, Gran.M15)
    candles = collector.grab(n)
    return candles

candles = get_candles(3)
for candle in candles:
    print(float(str(candle.bid.o))>1)


In [ ]:
def trading_job():
    candles = get_candles(3)
    dfstream = pd.DataFrame(columns=['Open','Close','High','Low'])
    
    i=0
    for candle in candles:
        dfstream.loc[i, ['Open']] = float(str(candle.bid.o))
        dfstream.loc[i, ['Close']] = float(str(candle.bid.c))
        dfstream.loc[i, ['High']] = float(str(candle.bid.h))
        dfstream.loc[i, ['Low']] = float(str(candle.bid.l))
        i=i+1

    dfstream['Open'] = dfstream['Open'].astype(float)
    dfstream['Close'] = dfstream['Close'].astype(float)
    dfstream['High'] = dfstream['High'].astype(float)
    dfstream['Low'] = dfstream['Low'].astype(float)

    signal = signal_generator(dfstream.iloc[:-1,:])#
    
    # EXECUTING ORDERS
    #accountID = "XXXXXXX" #your account ID here
    client = API(access_token)
         
    SLTPRatio = 2.
    previous_candleR = abs(dfstream['High'].iloc[-2]-dfstream['Low'].iloc[-2])
    
    SLBuy = float(str(candle.bid.o))-previous_candleR
    SLSell = float(str(candle.bid.o))+previous_candleR

    TPBuy = float(str(candle.bid.o))+previous_candleR*SLTPRatio
    TPSell = float(str(candle.bid.o))-previous_candleR*SLTPRatio
    
    print(dfstream.iloc[:-1,:])
    print(TPBuy, "  ", SLBuy, "  ", TPSell, "  ", SLSell)
    signal = 2
    #Sell
    if signal == 1:
        mo = MarketOrderRequest(instrument="EUR_USD", units=-1000, takeProfitOnFill=TakeProfitDetails(price=TPSell).data, stopLossOnFill=StopLossDetails(price=SLSell).data)
        r = orders.OrderCreate(accountID, data=mo.data)
        rv = client.request(r)
        print(rv)
    #Buy
    elif signal == 2:
        mo = MarketOrderRequest(instrument="EUR_USD", units=1000, takeProfitOnFill=TakeProfitDetails(price=TPBuy).data, stopLossOnFill=StopLossDetails(price=SLBuy).data)
        r = orders.OrderCreate(accountID, data=mo.data)
        rv = client.request(r)
        print(rv)

### 4 - Executing orders automatically with a scheduler

In [ ]:
trading_job()

#scheduler = BlockingScheduler()
#scheduler.add_job(trading_job, 'cron', day_of_week='mon-fri', hour='00-23', minute='1,16,31,46', start_date='2022-01-12 12:00:00', timezone='America/Chicago')
#scheduler.start()